In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from relife import Weibull
pd.options.plotting.backend = "plotly"

In [15]:
duree_construction = 3
duree_projet = 20
CAPEX = 1200000
OPEX = CAPEX * 3/100
taux_actualisation = 7

In [24]:
df = pd.DataFrame(columns=[k for k in range(-duree_construction+1,duree_projet)])
df.loc["Coef. actualisation"] = [(1+taux_actualisation/100)**k for k in df.columns]
df.loc["CAPEX"] = 0
df.loc["CAPEX",:0] = CAPEX/duree_construction
df.loc["OPEX"] = 0
df.loc["OPEX",1:] = OPEX
df.loc["TOTEX"] = df.loc["CAPEX"] + df.loc["OPEX"]
df.loc["TOTEX actualisés"] = df.loc["TOTEX"]/df.loc["Coef. actualisation"]
df.transpose().plot()

In [23]:
df

,-2,-1,0,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,19
Coef. actualisation,0.873439,0.934579,1.0,1.07,1.1449,1.225043,1.310796,1.402552,1.50073,1.605781,...,1.967151,2.104852,2.252192,2.409845,2.578534,2.759032,2.952164,3.158815,3.379932,3.616528
CAPEX,400000.000000,400000.000000,400000.0,0.00,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OPEX,0.000000,0.000000,0.0,36000.00,36000.0000,36000.000000,36000.000000,36000.000000,36000.00000,36000.000000,...,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000
TOTEX,400000.000000,400000.000000,400000.0,36000.00,36000.0000,36000.000000,36000.000000,36000.000000,36000.00000,36000.000000,...,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000


In [51]:
def LCOE_constant_condition(CAPEX,OpexCapex_rate,capacity,capacity_factor,rate,duration):
    return (CAPEX*capacity + OpexCapex_rate*CAPEX*capacity*(1+rate)/rate*((1+rate)**duration-1))*1000/(capacity*capacity_factor*(1+rate)/rate*((1+rate)**duration-1))

In [70]:
def VAN(rate,flux):
    n = flux.shape[0]
    return flux * np.geomspace(start = 1+rate, stop =(1+rate)**n,num=n)

def VAN_OPEX_constant(OPEX,rate,duree_projet):
    flux_OPEX = np.full(duree_projet,OPEX)
    return VAN(rate,flux_OPEX)

In [76]:
A = VAN_OPEX_constant(CAPEX * 3/100,7/100,20)
np.sum(A)

1579146.3642138243

## A.1

In [61]:
LCOE_constant_condition(
    CAPEX = 1000,
    OpexCapex_rate = 3.5/100,
    capacity = 500,
    capacity_factor= 1700,
    rate = 6.5/100,
    duration = 20
)

34.814358110082615

## A.2

In [44]:
LCOE_constant_condition(
    CAPEX = 1200,
    OpexCapex_rate = 0.03,
    capacity = 500,
    capacity_factor= 2200,
    rate = 0.065,
    duration = 20
)

29.55513206571296